# 5. Portfolio Simulation

Full Monte Carlo simulation of portfolio loss distribution.

## Contents
1. Portfolio Setup
2. Monte Carlo Simulation
3. Loss Distribution
4. Waterfall Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

from privatecredit.models import PortfolioAggregator
from privatecredit.models.portfolio_aggregator import WaterfallConfig

## 1. Portfolio Setup

In [ ]:
# Portfolio parameters
np.random.seed(42)

n_loans = 500
total_balance = 500_000_000  # $500M
avg_balance = total_balance / n_loans

# Loan balances (log-normal)
balances = np.exp(np.random.normal(np.log(avg_balance), 0.5, n_loans))
balances = balances / balances.sum() * total_balance  # Rescale

# PDs (beta distribution)
pds = np.random.beta(2, 50, n_loans)  # Mean ~4%

# LGDs (beta distribution)
lgds = np.random.beta(2, 5, n_loans)  # Mean ~29%

print(f"Portfolio size: ${total_balance/1e6:.0f}M")
print(f"Mean PD: {pds.mean():.2%}")
print(f"Mean LGD: {lgds.mean():.2%}")
print(f"Expected Loss: ${(balances * pds * lgds).sum()/1e6:.2f}M")

## 2. Monte Carlo Simulation

In [ ]:
# Simulate with asset correlation
n_simulations = 10000
rho = 0.15  # Asset correlation

# Vasicek model
portfolio_losses = []

for _ in range(n_simulations):
    # Systematic factor
    Z = np.random.normal()
    
    # Idiosyncratic factors
    epsilon = np.random.normal(size=n_loans)
    
    # Latent variable
    A = np.sqrt(rho) * Z + np.sqrt(1 - rho) * epsilon
    
    # Default threshold (from PD)
    threshold = stats.norm.ppf(pds)
    
    # Defaults occur where A < threshold
    defaults = (A < threshold).astype(float)
    
    # Loss = Balance * Default * LGD
    loss = (balances * defaults * lgds).sum()
    portfolio_losses.append(loss)

portfolio_losses = np.array(portfolio_losses)
print(f"Simulated {n_simulations} scenarios")

## 3. Loss Distribution

In [ ]:
# Loss statistics
print("Loss Distribution:")
print(f"  Mean Loss: ${portfolio_losses.mean()/1e6:.2f}M ({portfolio_losses.mean()/total_balance:.2%})")
print(f"  Std Dev: ${portfolio_losses.std()/1e6:.2f}M")
print(f"  VaR 95%: ${np.percentile(portfolio_losses, 95)/1e6:.2f}M")
print(f"  VaR 99%: ${np.percentile(portfolio_losses, 99)/1e6:.2f}M")
print(f"  CVaR 99%: ${portfolio_losses[portfolio_losses >= np.percentile(portfolio_losses, 99)].mean()/1e6:.2f}M")

In [ ]:
# Plot loss distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(portfolio_losses/1e6, bins=50, edgecolor='white', color='steelblue', density=True)
axes[0].axvline(portfolio_losses.mean()/1e6, color='red', linestyle='--', label='Mean')
axes[0].axvline(np.percentile(portfolio_losses, 99)/1e6, color='orange', linestyle='--', label='VaR 99%')
axes[0].set_xlabel('Loss ($M)')
axes[0].set_ylabel('Density')
axes[0].set_title('Portfolio Loss Distribution')
axes[0].legend()

# CDF
sorted_losses = np.sort(portfolio_losses)/1e6
cdf = np.arange(1, len(sorted_losses)+1) / len(sorted_losses)
axes[1].plot(sorted_losses, cdf, linewidth=2)
axes[1].axhline(0.95, color='orange', linestyle='--', alpha=0.7, label='95%')
axes[1].axhline(0.99, color='red', linestyle='--', alpha=0.7, label='99%')
axes[1].set_xlabel('Loss ($M)')
axes[1].set_ylabel('Cumulative Probability')
axes[1].set_title('Loss CDF')
axes[1].legend()

plt.tight_layout()
plt.show()

## 4. Waterfall Analysis

In [ ]:
# CLO tranche structure
tranches = {
    'AAA': {'attachment': 0.30, 'detachment': 1.00, 'spread': 0.012},
    'AA': {'attachment': 0.22, 'detachment': 0.30, 'spread': 0.018},
    'A': {'attachment': 0.16, 'detachment': 0.22, 'spread': 0.025},
    'BBB': {'attachment': 0.10, 'detachment': 0.16, 'spread': 0.035},
    'BB': {'attachment': 0.05, 'detachment': 0.10, 'spread': 0.060},
    'Equity': {'attachment': 0.00, 'detachment': 0.05, 'spread': 0.150}
}

# Calculate tranche losses for each simulation
tranche_losses = {name: [] for name in tranches}

for loss in portfolio_losses:
    loss_rate = loss / total_balance
    for name, params in tranches.items():
        att = params['attachment']
        det = params['detachment']
        tranche_size = det - att
        
        # Tranche loss
        if loss_rate <= att:
            t_loss = 0
        elif loss_rate >= det:
            t_loss = 1
        else:
            t_loss = (loss_rate - att) / tranche_size
        
        tranche_losses[name].append(t_loss)

for name in tranches:
    tranche_losses[name] = np.array(tranche_losses[name])

In [ ]:
# Tranche loss statistics
print("Tranche Loss Rates:")
for name in tranches:
    losses = tranche_losses[name]
    print(f"  {name:6s}: Mean={losses.mean():.4%}, VaR99={np.percentile(losses, 99):.4%}")

In [ ]:
# Plot tranche loss distributions
fig, ax = plt.subplots(figsize=(12, 6))

positions = range(len(tranches))
names = list(tranches.keys())
means = [tranche_losses[n].mean() for n in names]
var99 = [np.percentile(tranche_losses[n], 99) for n in names]

ax.bar(positions, means, width=0.4, label='Mean Loss', color='steelblue')
ax.bar([p + 0.4 for p in positions], var99, width=0.4, label='VaR 99%', color='coral')
ax.set_xticks([p + 0.2 for p in positions])
ax.set_xticklabels(names)
ax.set_ylabel('Loss Rate')
ax.set_title('Tranche Loss Distribution')
ax.legend()
plt.tight_layout()
plt.show()

## Summary

- Monte Carlo with Vasicek correlation model
- Portfolio loss distribution with VaR/CVaR
- Waterfall allocates losses to tranches
- Senior tranches protected by subordination

**Next:** Stress testing (Notebook 06)